### Introduction
We will analyze a database with information on books, publishers, authors, and customer ratings and book reviews in order to find a value proposition to develop a new application for book lovers.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
tables = ['books','authors','publishers','ratings','reviews']
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5;"
    df = pd.io.sql.read_sql(query, con=engine)
    print(f"--- {table} ---")
    display(df)   

--- books ---


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


--- authors ---


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


--- publishers ---


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


--- ratings ---


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


--- reviews ---


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [3]:
query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'books'
ORDER BY ordinal_position;
"""
pd.read_sql(query, con=engine)

,column_name,data_type
0,book_id,integer
1,author_id,integer
2,title,text
3,num_pages,integer
4,publication_date,date
5,publisher_id,integer


In [4]:
query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'authors'
ORDER BY ordinal_position;
"""
pd.read_sql(query, con=engine)

,column_name,data_type
0,author_id,integer
1,author,text


In [5]:
query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'publishers'
ORDER BY ordinal_position;
"""
pd.read_sql(query, con=engine)

,column_name,data_type
0,publisher_id,integer
1,publisher,text


In [6]:
query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'ratings'
ORDER BY ordinal_position;
"""
pd.read_sql(query, con=engine)

,column_name,data_type
0,rating_id,integer
1,book_id,integer
2,username,text
3,rating,integer


In [7]:
query = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'reviews'
ORDER BY ordinal_position;
"""
pd.read_sql(query, con=engine)

,column_name,data_type
0,review_id,integer
1,book_id,integer
2,username,text
3,text,text


In [8]:
query = """
SELECT
    COUNT(*) AS books_after_2000
FROM
    books
WHERE
    publication_date > '2000-01-01';
"""

books_after_2000 = pd.read_sql(query, con=engine)
display(books_after_2000)

,books_after_2000
0,819


#### After 2000, 819 books were published

In [34]:
query = """
SELECT
    ratings.book_id,
    COUNT(DISTINCT reviews.review_id) AS no_reviews,
    ROUND(AVG(ratings.rating), 1) AS avg_rating
FROM
    reviews 
JOIN
    ratings 
ON
    reviews.book_id = ratings.book_id
GROUP BY
    ratings.book_id
ORDER BY
    avg_rating DESC;
"""

reviews_ratings = pd.read_sql(query, con=engine)

display(reviews_ratings.head())

,book_id,no_reviews,avg_rating
0,20,2,5.0
1,421,2,5.0
2,62,2,5.0
3,967,2,5.0
4,224,2,5.0


#### Although there are several books with a 5-star rating, it is also important to consider the number of reviews.

In [36]:
query = """
SELECT
    publishers.publisher,
    COUNT(books.book_id) AS over_50_pages
    
FROM
    books
JOIN
    publishers
ON 
    books.publisher_id = publishers.publisher_id

WHERE
    books.num_pages > 50
    
GROUP BY
    publishers.publisher_id
    
ORDER BY
    over_50_pages DESC;
"""


over_50_pages = pd.read_sql(query, con=engine)
display(over_50_pages.head())

,publisher,over_50_pages
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


#### Penguin Books is the publisher with the most published books

In [40]:
query = """
SELECT 
    ROUND(AVG(text_reviews_per_user), 2) AS avg_text_reviews_per_user
FROM (
SELECT
    ratings.username,
    COUNT(DISTINCT reviews.review_id) AS text_reviews_per_user
    
FROM
    ratings
LEFT JOIN
    reviews ON ratings.username = reviews.username

WHERE 
    reviews.text IS NOT NULL 
    AND TRIM(reviews.text) <> ''
    
GROUP BY
    ratings.username

HAVING 
    COUNT(DISTINCT ratings.book_id) > 50
) AS user_reviews;
"""


avg_text_reviews_per_user = pd.read_sql(query, con=engine)
display(avg_text_reviews_per_user.head())

,avg_text_reviews_per_user
0,24.33


#### On average, users who rated more than 50 books wrote text reviews for about 24 of them.